In [4]:
!pip install gradio

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/54.1 MB ? eta -:--:--
   ---------------------------------------- 0.3/54.1 MB ? eta -:--:--
    --------------------------------------- 0.8/54.1 MB 3.0 MB/s eta 0:00:18
    --------------------------------------- 1.3/54.1 MB 2.6 MB/s eta 0:00:21
   - -------------------------------------- 1.6/54.1 MB 2.2 MB/s eta 0:00:25
   - -------------------------------------- 1.8/54.1 MB 1.9 MB/s eta 0:00:28
   - -------------------------------------- 2.1/54.1 MB 1.7 MB/s eta 0:00:31
   - -------------------------------------- 2.4/54.1 MB 1.7 MB/s eta 0:00:31
   - -------------------------------------- 2.6/54.1 MB 1.7 MB/s eta 0:00:32
   -- ------------------------------------- 2.9/54.1 MB 1.6 MB/s eta 0:00:33
   -- ------------------------------------- 3.1/54.1 MB 1.5 MB/s eta 0:00:34
   -- ------------------------------------- 3.4/54.1 MB 1.5 MB/s eta 0:00:34
   -- ------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.5.1 requires ipython!=8.17.1,<9.0.0,>=8.13.0; python_version > "3.8", but you have ipython 7.34.0 which is incompatible.


In [6]:
# Import pandas
import pandas as pd
# Import the required dependencies from sklearn
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

# Set the column width to view the text message data.
pd.set_option('max_colwidth', 200)

# Import Gradio
import gradio as gr

In [7]:
def sms_classification(sms_text_df):
    """
    Perform SMS classification using a pipeline with TF-IDF vectorization and Linear Support Vector Classification.

    Parameters:
    - sms_text_df (pd.DataFrame): DataFrame containing 'text_message' and 'label' columns for SMS classification.

    Returns:
    - text_clf (Pipeline): Fitted pipeline model for SMS classification.

    This function takes a DataFrame with 'text_message' and 'label' columns, splits the data into
    training and testing sets, builds a pipeline with TF-IDF vectorization and Linear Support Vector
    Classification, and fits the model to the training data. 
    The fitted pipeline is returned to make future predictions.
    """
    # Set the features variable to the text message column.
    features = sms_text_df['text_message']
    # Set the target variable to the "label" column.
    target = sms_text_df['label'] 

    # Split data into training and testing and set the test_size = 33%
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.33, random_state=42)

    # Build a pipeline to transform the test set to compare to the training set.
    text_clf = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('clf', LinearSVC(dual='auto'))  # Setting the dual parameter explicitly
    ])

    # Fit the model to the transformed training data and return model.
    text_clf.fit(X_train, y_train)
    return text_clf

In [10]:
# Load the dataset into a DataFrame
sms_text_df = pd.read_csv("Resources/SMSSpamCollection.csv", sep=',')
sms_text_df.head()

,label,text_message
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"


In [16]:
# Call the sms_classification function with the DataFrame and set the result to the "text_clf" variable
text_clf = sms_classification(sms_text_df)

In [25]:
# Create a function called `sms_prediction` that takes in the SMS text and predicts the whether the text is "not spam" or "spam". 
# The function should return the SMS message, and say whether the text is "not spam" or "spam".
def sms_prediction(text):
    """
    Predict the spam/ham classification of a given text message using a pre-trained model.

    Parameters:
    - text (str): The text message to be classified.

    Returns:
    - str: A message indicating whether the text message is classified as spam or not.

    This function takes a text message and a pre-trained pipeline model, then predicts the
    spam/ham classification of the text. The result is a message stating whether the text is
    classified as spam or not.
    """
    # Create a variable that will hold the prediction of a new text.
    prediction = text_clf.predict([text])[0] 
    # Using a conditional if the prediction is "ham" return the message:
    # f'The text message: "{text}", is not spam.' Else, return f'The text message: "{text}", is spam.'
    if prediction == 'ham':
        return f'The text message: "{text}", is not spam.'
    else:
        return f'The text message: "{text}", is spam.'
    

In [26]:
# Create a sms_app that takes a textbox for the inputs and has a textbox for the output.  
# Povide labels for each textbox. 

    
# Launch the app.


def sms_prediction_interface():
    """
    Create and launch a Gradio Interface application for SMS spam detection.

    This function creates a Gradio Interface application that takes a text input for SMS messages
    and provides a prediction of whether the message is spam or not.
    """
    # Define the input and output components for the Gradio Interface
    input_textbox = gr.Textbox(lines=2, placeholder="Enter your SMS text here...", label="Input SMS Text")
    output_textbox = gr.Textbox(label="Prediction")

    # Create the Gradio Interface application with the defined inputs, outputs, and the prediction function
    sms_app = gr.Interface(fn=sms_prediction, inputs=input_textbox, outputs=output_textbox, 
                           title="SMS Spam Detection", description="Enter an SMS text to check if it is spam or not.")

    # Launch the app with a public link
    sms_app.launch(share=True)

# Call function to create and launch the Gradio Interface application
sms_prediction_interface()


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://2546a273720e19d829.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Test the following text messages. 

---

1. You are a lucky winner of $5000!
2. You won 2 free tickets to the Super Bowl.
3. You won 2 free tickets to the Super Bowl text us to claim your prize.
4. Thanks for registering. Text 4343 to receive free updates on medicare.